In [1]:
from icecube import dataio, icetray , dataclasses 

In [2]:
import pandas as pd
pd.set_option("display.max_columns", None)   

In [3]:
from collections.abc import Iterable

In [4]:
GCD_PATH = "/project/6008051/pone_simulation/GCD_Library/PONE_800mGrid.i3.gz"
data_file = dataio.I3File(GCD_PATH)
geometry = data_file.pop_frame()

In [5]:
# dir(geometry)

In [6]:
print(type(geometry))
print("Stop:", geometry.Stop)   
print("Key's:", list(geometry.keys()))

<class 'icecube._icetray.I3Frame'>
Stop: Geometry
Key's: ['Subdetectors', 'StartTime', 'I3OMGeoMap', 'I3ModuleGeoMap', 'I3Geometry', 'EndTime']


In [7]:
# Small helper: print the type and a short summary for each key in the frame
for key in geometry.keys():
    obj = geometry[key]
    print("="*60)
    print(f"Key: {key}")
    print("Type:", type(obj))
    try:
        print("dir(obj):", dir(obj))
    except Exception as e:
        print("dir(obj) could not be obtained:", e)

Key: Subdetectors
Type: <class 'icecube._dataclasses.I3MapModuleKeyString'>
dir(obj): ['__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__getstate_manages_dict__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__instance_size__', '__iter__', '__key_type__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__safe_for_unpickling__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__value_type__', '__weakref__', 'clear', 'from_frame', 'get', 'items', 'keys', 'pop', 'popitem', 'update', 'values']
Key: StartTime
Type: <class 'icecube._dataclasses.I3Time'>
dir(obj): ['Apr', 'Aug', 'BadMonth', 'BadWeekday', 'Dec', 'Feb', 'Friday', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Monday', 'Month', 'Nov', 'Oct', 'Saturday', 'Sep', 'Sunday'

WARN (dataclasses): Reading version 0 from file into version 1 of I3ModuleGeo class, all modules are assumed spherical. (I3ModuleGeo.cxx:75 in void I3ModuleGeo::serialize(Archive&, unsigned int) [with Archive = icecube::archive::portable_binary_iarchive])
WARN (dataclasses): Reading version 0 from file into version 1 of I3ModuleGeo class, all modules are assumed spherical. (I3ModuleGeo.cxx:75 in void I3ModuleGeo::serialize(Archive&, unsigned int) [with Archive = icecube::archive::portable_binary_iarchive])
WARN (dataclasses): Reading version 0 from file into version 1 of I3ModuleGeo class, all modules are assumed spherical. (I3ModuleGeo.cxx:75 in void I3ModuleGeo::serialize(Archive&, unsigned int) [with Archive = icecube::archive::portable_binary_iarchive])
WARN (dataclasses): Reading version 0 from file into version 1 of I3ModuleGeo class, all modules are assumed spherical. (I3ModuleGeo.cxx:75 in void I3ModuleGeo::serialize(Archive&, unsigned int) [with Archive = icecube::archive::por

# - Subdetectors
##### (no useful information here)

In [8]:
subdets = geometry["Subdetectors"]
print(type(subdets))

<class 'icecube._dataclasses.I3MapModuleKeyString'>


In [9]:
sample_key = next(iter(subdets.keys()))
attrs = [a for a in dir(sample_key) if not a.startswith("_")]

rows = []

for mkey in subdets.keys():
    name = subdets[mkey]   

    row = {"subdetector_name": str(name)}

    # add simple attributes from ModuleKey
    for attr in attrs:
        v = getattr(mkey, attr)
        if isinstance(v, (int, float, str, bool)):
            row[attr] = v

    rows.append(row)

subdet_df = pd.DataFrame(rows)
subdet_df


,subdetector_name,om,string
0,Upgrade,1,1
1,Upgrade,2,1
2,Upgrade,3,1
3,Upgrade,4,1
4,Upgrade,5,1
...,...,...,...
6795,Upgrade,16,340
6796,Upgrade,17,340
6797,Upgrade,18,340
6798,Upgrade,19,340


In [10]:
len(subdets)
# number of OM

6800

In [11]:
# print(subdets)
# (name of these subdetectors: Upgrade)

# ModuleKey has information of StringId and OM Id

# - StartTime and EndTime
###### (no useful information here)

In [12]:
start = geometry["StartTime"]
end   = geometry["EndTime"]
print(start)
print(end)

2019-01-01 00:00:00.000,000,000,0 UTC
1902-11-25 17:31:44.000,000,000,0 UTC


In [13]:
start

I3Time(2019,                 0L)

In [14]:
end

I3Time(2039,                 0L)

In [15]:
print(repr(start))
print(str(start))


I3Time(2019,                 0L)
2019-01-01 00:00:00.000,000,000,0 UTC


In [16]:
print(repr(end))
print(str(end))


I3Time(2039,                 0L)
1902-11-25 17:31:44.000,000,000,0 UTC


# - I3OMGeoMap
#####  position information available, but it is on OM level, not PMT level: same position for all PMTs in one OM

In [17]:
om_map = geometry["I3OMGeoMap"]
print(type(om_map))
print("Number of OMs:", len(om_map))

# 340 * 20 * 16

<class 'icecube._dataclasses.I3OMGeoMap'>
Number of OMs: 108800


In [18]:
# om_map.keys()
# OMKey has information of String Id, OM Id, PMT Id

In [19]:
# for k in list(om_map.keys())[:5]:
#    print(k)

In [20]:
# for k in list(om_map.values())[:5]:
#    print(k)


In [21]:
sample_key, sample_geo = next(iter(om_map.items()))
attrs = [a for a in dir(sample_geo) if not a.startswith("_")]

# we handle these manually 
skip = {"position", "orientation", "omtype", "area"}
attrs = [a for a in attrs if a not in skip]

rows = []

for omkey, omgeo in om_map.items():
    pos = omgeo.position
    ori = omgeo.orientation

    row = {
        "string": omkey.string,
        "om": omkey.om,
        "pmt": getattr(omkey, "pmt", None),

        "x": pos.x,
        "y": pos.y,
        "z": pos.z,

        "dir_x": ori.dir.x,
        "dir_y": ori.dir.y,
        "dir_z": ori.dir.z,

        "up_x": ori.up.x,
        "up_y": ori.up.y,
        "up_z": ori.up.z,

        "right_x": ori.right.x,
        "right_y": ori.right.y,
        "right_z": ori.right.z,

        "omtype": int(omgeo.omtype),
        "area": float(omgeo.area),
    }

    # add all other simple attributes from omgeo
    for name in attrs:
        v = getattr(omgeo, name)
        if isinstance(v, (int, float, str, bool)):
            row[name] = v

    rows.append(row)

om_df = pd.DataFrame(rows)
om_df.head()


,string,om,pmt,x,y,z,dir_x,dir_y,dir_z,up_x,up_y,up_z,right_x,right_y,right_z,omtype,area,AMANDA,AbaloneHub,AcousticEmitter,DEgg,DMIce,FOM,FibreComm,IceAct,IceCube,IceTop,LOM,LOM16,LOM18,PDOM,POCAM,PencilBeam,RadioEmitter,RadioReceiver,Scintillator,UnknownType,WOM,isoPDOM,mDOM
0,1,1,1,-120.0,-730.717968,-450.0,1.224647e-16,0.0,-1.0,1.0,0.0,6.123234e-17,6.123234e-17,-1.0,6.123234e-17,130,0.585754,10,240,230,120,160,150,250,50,20,30,170,171,172,110,200,210,220,180,40,0,140,111,130
1,1,1,2,-120.0,-730.717968,-450.0,1.224647e-16,0.0,-1.0,1.0,0.0,6.123234e-17,6.123234e-17,-1.0,6.123234e-17,130,0.585754,10,240,230,120,160,150,250,50,20,30,170,171,172,110,200,210,220,180,40,0,140,111,130
2,1,1,3,-120.0,-730.717968,-450.0,1.224647e-16,0.0,-1.0,1.0,0.0,6.123234e-17,6.123234e-17,-1.0,6.123234e-17,130,0.585754,10,240,230,120,160,150,250,50,20,30,170,171,172,110,200,210,220,180,40,0,140,111,130
3,1,1,4,-120.0,-730.717968,-450.0,1.224647e-16,0.0,-1.0,1.0,0.0,6.123234e-17,6.123234e-17,-1.0,6.123234e-17,130,0.585754,10,240,230,120,160,150,250,50,20,30,170,171,172,110,200,210,220,180,40,0,140,111,130
4,1,1,5,-120.0,-730.717968,-450.0,1.224647e-16,0.0,-1.0,1.0,0.0,6.123234e-17,6.123234e-17,-1.0,6.123234e-17,130,0.585754,10,240,230,120,160,150,250,50,20,30,170,171,172,110,200,210,220,180,40,0,140,111,130


In [22]:
len(om_df)

108800

In [23]:
# Columns that have the same value in all rows
const_cols = om_df.columns[om_df.nunique(dropna=False) == 1]
print(const_cols)


Index(['dir_x', 'dir_y', 'dir_z', 'up_x', 'up_y', 'up_z', 'right_x', 'right_y',
       'right_z', 'omtype', 'area', 'AMANDA', 'AbaloneHub', 'AcousticEmitter',
       'DEgg', 'DMIce', 'FOM', 'FibreComm', 'IceAct', 'IceCube', 'IceTop',
       'LOM', 'LOM16', 'LOM18', 'PDOM', 'POCAM', 'PencilBeam', 'RadioEmitter',
       'RadioReceiver', 'Scintillator', 'UnknownType', 'WOM', 'isoPDOM',
       'mDOM'],
      dtype='object')


In [24]:
# those same values:
print(om_df[const_cols].iloc[0])


dir_x              1.224647e-16
dir_y              0.000000e+00
dir_z             -1.000000e+00
up_x               1.000000e+00
up_y               0.000000e+00
up_z               6.123234e-17
right_x            6.123234e-17
right_y           -1.000000e+00
right_z            6.123234e-17
omtype             1.300000e+02
area               5.857538e-01
AMANDA             1.000000e+01
AbaloneHub         2.400000e+02
AcousticEmitter    2.300000e+02
DEgg               1.200000e+02
DMIce              1.600000e+02
FOM                1.500000e+02
FibreComm          2.500000e+02
IceAct             5.000000e+01
IceCube            2.000000e+01
IceTop             3.000000e+01
LOM                1.700000e+02
LOM16              1.710000e+02
LOM18              1.720000e+02
PDOM               1.100000e+02
POCAM              2.000000e+02
PencilBeam         2.100000e+02
RadioEmitter       2.200000e+02
RadioReceiver      1.800000e+02
Scintillator       4.000000e+01
UnknownType        0.000000e+00
WOM     

In [25]:
om_df = om_df.loc[:, om_df.nunique(dropna=False) > 1]


In [26]:
om_df

,string,om,pmt,x,y,z
0,1,1,1,-120.0,-730.717968,-450.0
1,1,1,2,-120.0,-730.717968,-450.0
2,1,1,3,-120.0,-730.717968,-450.0
3,1,1,4,-120.0,-730.717968,-450.0
4,1,1,5,-120.0,-730.717968,-450.0
...,...,...,...,...,...,...
108795,340,20,12,240.0,724.204711,500.0
108796,340,20,13,240.0,724.204711,500.0
108797,340,20,14,240.0,724.204711,500.0
108798,340,20,15,240.0,724.204711,500.0


In [27]:
# the x,y,z here are the positions of the Optical Module, not the PMT.
# all PMTs inside a OM have the same position and the same orientation(?). read these once again
# ask Rasmus

In [28]:
om_df["pos_tuple"] = list(
    zip(om_df["x"].round(6), om_df["y"].round(6), om_df["z"].round(6))
)

uniq_per_om = om_df.groupby(["string", "om"])["pos_tuple"].nunique()

print("Max distinct positions per (string, om):", uniq_per_om.max())

if uniq_per_om.max() == 1:
    print("=> Positions are OM-level (same for all PMTs within an OM).")
else:
    print("=> Positions are PMT-level (they vary between PMTs in the same OM).")

example_key = uniq_per_om.idxmax()
print("\nExample (string, om):", example_key)
print(om_df[(om_df["string"] == example_key[0]) & (om_df["om"] == example_key[1])])


Max distinct positions per (string, om): 1
=> Positions are OM-level (same for all PMTs within an OM).

Example (string, om): (1, 1)
    string  om  pmt      x           y      z                      pos_tuple
0        1   1    1 -120.0 -730.717968 -450.0  (-120.0, -730.717968, -450.0)
1        1   1    2 -120.0 -730.717968 -450.0  (-120.0, -730.717968, -450.0)
2        1   1    3 -120.0 -730.717968 -450.0  (-120.0, -730.717968, -450.0)
3        1   1    4 -120.0 -730.717968 -450.0  (-120.0, -730.717968, -450.0)
4        1   1    5 -120.0 -730.717968 -450.0  (-120.0, -730.717968, -450.0)
5        1   1    6 -120.0 -730.717968 -450.0  (-120.0, -730.717968, -450.0)
6        1   1    7 -120.0 -730.717968 -450.0  (-120.0, -730.717968, -450.0)
7        1   1    8 -120.0 -730.717968 -450.0  (-120.0, -730.717968, -450.0)
8        1   1    9 -120.0 -730.717968 -450.0  (-120.0, -730.717968, -450.0)
9        1   1   10 -120.0 -730.717968 -450.0  (-120.0, -730.717968, -450.0)
10       1   1   11 

# - I3ModuleGeoMap

In [29]:
mod_map = geometry["I3ModuleGeoMap"]
print(type(mod_map))
print("Number of modules:", len(mod_map))

# 340 * 20

<class 'icecube._dataclasses.I3ModuleGeoMap'>
Number of modules: 6800


In [30]:
# print(mod_map)

In [31]:
# discover attrs once
k0, g0 = next(iter(mod_map.items()))
key_attrs = [a for a in dir(k0) if not a.startswith("_")]
geo_attrs = [a for a in dir(g0) if not a.startswith("_")]

rows = []

for mkey, mgeo in mod_map.items():
    # try both 'pos' and 'position'
    pos = getattr(mgeo, "pos", None)
    if pos is None:
        pos = getattr(mgeo, "position", None)

    ori = getattr(mgeo, "orientation", None)
    d   = getattr(mgeo, "dir", None)

    row = {}

    # ---- ModuleKey info ----
    for a in key_attrs:
        v = getattr(mkey, a)
        if isinstance(v, (int, float, str, bool)):
            row[f"key_{a}"] = v

    # ---- position ----
    if pos is not None:
        row["x"] = pos.x
        row["y"] = pos.y
        row["z"] = pos.z

    # ---- direction vector ----
    if d is not None:
        row["dir_x"] = d.x
        row["dir_y"] = d.y
        row["dir_z"] = d.z
        # if zenith/azimuth exist:
        for name in ("zenith", "azimuth"):
            if hasattr(d, name):
                row[f"dir_{name}"] = getattr(d, name)

    # ---- orientation vectors ----
    if ori is not None:
        for base, vec in (("up", ori.up), ("right", ori.right)):
            row[f"{base}_x"] = vec.x
            row[f"{base}_y"] = vec.y
            row[f"{base}_z"] = vec.z

    # ---- all other scalar attrs of mgeo ----
    for a in geo_attrs:
        if a in {"pos", "position", "orientation", "dir"}:
            continue
        if a in row:  # don't overwrite existing keys
            continue
        v = getattr(mgeo, a)
        if isinstance(v, (int, float, str, bool)):
            row[a] = v

    rows.append(row)

mod_df = pd.DataFrame(rows)
mod_df.head()


,key_om,key_string,x,y,z,dir_x,dir_y,dir_z,dir_zenith,dir_azimuth,up_x,up_y,up_z,right_x,right_y,right_z,AMANDA,AbaloneHub,AcousticEmitter,DEgg,DMIce,FOM,FibreComm,IceAct,IceCube,IceTop,LOM,LOM16,LOM18,PDOM,POCAM,PencilBeam,RadioEmitter,RadioReceiver,Scintillator,UnknownType,WOM,height,mDOM,module_type,radius,rr,rz
0,1,1,-120.0,-730.717968,-450.0,1.224647e-16,0.0,-1.0,0.0,3.141593,1.0,0.0,6.123234e-17,6.123234e-17,-1.0,6.123234e-17,10,240,230,120,160,150,250,50,20,30,170,171,172,110,200,210,220,180,45,0,140,NaN,40,40,0.2159,0.2159,0.2159
1,2,1,-120.0,-730.717968,-400.0,1.224647e-16,0.0,-1.0,0.0,3.141593,1.0,0.0,6.123234e-17,6.123234e-17,-1.0,6.123234e-17,10,240,230,120,160,150,250,50,20,30,170,171,172,110,200,210,220,180,45,0,140,NaN,40,40,0.2159,0.2159,0.2159
2,3,1,-120.0,-730.717968,-350.0,1.224647e-16,0.0,-1.0,0.0,3.141593,1.0,0.0,6.123234e-17,6.123234e-17,-1.0,6.123234e-17,10,240,230,120,160,150,250,50,20,30,170,171,172,110,200,210,220,180,45,0,140,NaN,40,40,0.2159,0.2159,0.2159
3,4,1,-120.0,-730.717968,-300.0,1.224647e-16,0.0,-1.0,0.0,3.141593,1.0,0.0,6.123234e-17,6.123234e-17,-1.0,6.123234e-17,10,240,230,120,160,150,250,50,20,30,170,171,172,110,200,210,220,180,45,0,140,NaN,40,40,0.2159,0.2159,0.2159
4,5,1,-120.0,-730.717968,-250.0,1.224647e-16,0.0,-1.0,0.0,3.141593,1.0,0.0,6.123234e-17,6.123234e-17,-1.0,6.123234e-17,10,240,230,120,160,150,250,50,20,30,170,171,172,110,200,210,220,180,45,0,140,NaN,40,40,0.2159,0.2159,0.2159


In [32]:
mod_df.shape        


(6800, 43)

In [33]:
unique_counts = mod_df.nunique(dropna=False)

constant_cols = unique_counts[unique_counts == 1].index.tolist()
varying_cols  = unique_counts[unique_counts > 1].index.tolist()

print("Columns that are constant across all rows:")
for c in constant_cols:
    print("  -", c)

print("\nColumns that vary:")
for c in varying_cols:
    print("  -", c)


Columns that are constant across all rows:
  - dir_x
  - dir_y
  - dir_z
  - dir_zenith
  - dir_azimuth
  - up_x
  - up_y
  - up_z
  - right_x
  - right_y
  - right_z
  - AMANDA
  - AbaloneHub
  - AcousticEmitter
  - DEgg
  - DMIce
  - FOM
  - FibreComm
  - IceAct
  - IceCube
  - IceTop
  - LOM
  - LOM16
  - LOM18
  - PDOM
  - POCAM
  - PencilBeam
  - RadioEmitter
  - RadioReceiver
  - Scintillator
  - UnknownType
  - WOM
  - height
  - mDOM
  - module_type
  - radius
  - rr
  - rz

Columns that vary:
  - key_om
  - key_string
  - x
  - y
  - z


In [34]:
mod_df = mod_df.loc[:, mod_df.nunique(dropna=False) > 1]


In [35]:
mod_df

,key_om,key_string,x,y,z
0,1,1,-120.0,-730.717968,-450.0
1,2,1,-120.0,-730.717968,-400.0
2,3,1,-120.0,-730.717968,-350.0
3,4,1,-120.0,-730.717968,-300.0
4,5,1,-120.0,-730.717968,-250.0
...,...,...,...,...,...
6795,16,340,240.0,724.204711,300.0
6796,17,340,240.0,724.204711,350.0
6797,18,340,240.0,724.204711,400.0
6798,19,340,240.0,724.204711,450.0


In [36]:
#### do all have the same orientation???
#### how about the orientation of PMTs

# - I3Geometry


In [37]:
geo = geometry["I3Geometry"]

print("Type:", type(geo))
print("Attributes:")
print(dir(geo))


Type: <class 'icecube._dataclasses.I3Geometry'>
Attributes:
['__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__getstate_manages_dict__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__instance_size__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__safe_for_unpickling__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'antennageo', 'end_time', 'from_frame', 'iceactgeo', 'omgeo', 'scintgeo', 'snow_height_provenance', 'start_time', 'stationgeo', 'tankgeo']


In [38]:
print(geo)

In [39]:
# All attributes
all_attrs = dir(geo)

# Non-dunder attributes
public_attrs = [a for a in all_attrs if not a.startswith("_")]

print("Public attrs:")
for a in public_attrs:
    print("  -", a)


Public attrs:
  - antennageo
  - end_time
  - from_frame
  - iceactgeo
  - omgeo
  - scintgeo
  - snow_height_provenance
  - start_time
  - stationgeo
  - tankgeo


In [40]:
rows = []
for a in public_attrs:
    value = getattr(geo, a)
    # try to get length if it behaves like a container
    try:
        n = len(value)
    except TypeError:
        n = None

    rows.append({
        "attr": a,
        "type": type(value).__name__,
        "len": n,
        "repr_sample": repr(value),  
    })

geo_attrs_df = pd.DataFrame(rows)
geo_attrs_df


,attr,type,len,repr_sample
0,antennageo,I3AntennaGeoMap,0.0,<icecube._dataclasses.I3AntennaGeoMap object a...
1,end_time,I3Time,NaN,"I3Time(2039, 0L)"
2,from_frame,function,NaN,<Boost.Python.function object at 0x2ee38d0>
3,iceactgeo,I3IceActGeoMap,0.0,<icecube._dataclasses.I3IceActGeoMap object at...
4,omgeo,I3OMGeoMap,108800.0,<icecube._dataclasses.I3OMGeoMap object at 0x1...
5,scintgeo,I3ScintGeoMap,0.0,<icecube._dataclasses.I3ScintGeoMap object at ...
6,snow_height_provenance,SnowHeightProvenance,NaN,icecube._dataclasses.SnowHeightProvenance.Unknown
7,start_time,I3Time,NaN,"I3Time(2019, 0L)"
8,stationgeo,I3StationGeoMap,0.0,<icecube._dataclasses.I3StationGeoMap object a...
9,tankgeo,method,NaN,<bound method tankgeo of <icecube._dataclasses...


In [41]:
for name in public_attrs:
    val = getattr(geo, name)

    print(f"\n{name}")
    print("  type:", type(val))

    # try to show some size info
    if hasattr(val, "keys"):
        print("  n_keys:", len(val))
    elif hasattr(val, "__len__") and not isinstance(val, (str, bytes)):
        print("  len:", len(val))

    # small repr preview
    print("  repr:", repr(val)[:200])



antennageo
  type: <class 'icecube._dataclasses.I3AntennaGeoMap'>
  n_keys: 0
  repr: <icecube._dataclasses.I3AntennaGeoMap object at 0x147f12242260>

end_time
  type: <class 'icecube._dataclasses.I3Time'>
  repr: I3Time(2039,                 0L)

from_frame
  type: <class 'Boost.Python.function'>
  repr: <Boost.Python.function object at 0x2ee38d0>

iceactgeo
  type: <class 'icecube._dataclasses.I3IceActGeoMap'>
  n_keys: 0
  repr: <icecube._dataclasses.I3IceActGeoMap object at 0x147f122423b0>

omgeo
  type: <class 'icecube._dataclasses.I3OMGeoMap'>
  n_keys: 108800
  repr: <icecube._dataclasses.I3OMGeoMap object at 0x147f12242260>

scintgeo
  type: <class 'icecube._dataclasses.I3ScintGeoMap'>
  n_keys: 0
  repr: <icecube._dataclasses.I3ScintGeoMap object at 0x147f122423b0>

snow_height_provenance
  type: <class 'icecube._dataclasses.SnowHeightProvenance'>
  repr: icecube._dataclasses.SnowHeightProvenance.Unknown

start_time
  type: <class 'icecube._dataclasses.I3Time'>
  repr: I3Time

In [42]:
om_map  = geometry["I3OMGeoMap"]
geo     = geometry["I3Geometry"]
om_map2 = geo.omgeo

print(len(om_map), len(om_map2))

print("same keys? :", set(om_map.keys()) == set(om_map2.keys()))

keys = list(om_map.keys())
print("all values equal? :",
      all(om_map[k] == om_map2[k] for k in keys))


108800 108800
same keys? : True
all values equal? : True


In [43]:
print(om_map is om_map2)


False
